In [100]:
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn import *
from sklearn.metrics import roc_curve, auc

np.random.seed(0)
%matplotlib inline

X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=',')

# X,Y = ml.shuffleData(X,Y)
# Fetureset1_X = X
X = X[:,0:40]
Xtr, Xva, Ytr, Yva = ml.splitData(X, Y, 0.80)

Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')
Xte = Xte[:,:40]

Xtr, param = ml.transforms.rescale(Xtr)
Xva, _ = ml.transforms.rescale(Xva,param)
Xte, _ = ml.transforms.rescale(Xte,param)

In [ ]:
# print(X.shape)
# print(Y.shape)
# print(Xte.shape)

# Fetureset1_X = X[:,0:40]
# Fetureset2_X = X[:,40:69]
# Fetureset3_X = X[:,69:107]

# print(Fetureset1_X.shape)
# print(Fetureset2_X.shape)
# print(Fetureset3_X.shape)
# print(X[:,0:40])
# print(X[:,41:69])
# print(X[:,70:108])
# # for i in X[:1]:
# #     for j in i :
# #         x += 1
# #         print(x,j)


In [101]:

nBag = 100
classifiers2 = [ None ] * nBag

mt,nt = Xte.shape

Ye = np.zeros((mt,2))
auc_list = []

for i in range(nBag):
    Xi,Yi = ml.bootstrapData(X,Y,mt) 
    classifiers2[i] = ml.dtree.treeClassify(Xi, Yi, maxDepth = 25 , minLeaf = 4, minParent = 512, nFeatures = 41)
    #Train a model on data Xi,Yi
    auc_list.append(classifiers2[i].auc(Xi,Yi))
    #Train a model on data Xi,Yi
    Ye += classifiers2[i].predictSoft(Xte)
learner = Ye / 25
print(np.mean(auc_list))

0.7677337912345241


In [21]:

nBag = 1000
classifiers2 = [ None ] * nBag

mt,nt = Xte.shape

Ye = np.zeros((mt,2))
auc_list = []

for i in range(nBag):
    Xi,Yi = ml.bootstrapData(X,Y,mt) 
    classifiers2[i] = ml.dtree.treeClassify(Xi, Yi, maxDepth = 25 , minLeaf = 4, minParent = 512, nFeatures = 60)
    #Train a model on data Xi,Yi
    auc_list.append(classifiers2[i].auc(Xi,Yi))
    #Train a model on data Xi,Yi
    Ye += classifiers2[i].predictSoft(Xte)
learner = Ye / 25
print(np.mean(auc_list))

0.7764267277353187


In [27]:

Yte = np.vstack((np.arange(Xte.shape[0]), learner[:,1])).T
# Output a file with two columns, a row ID and a confidence in class 1:
np.savetxt('Y_submit2.txt',Yte,'%d, %.2f',header='Id,Predicted',delimiter=',')
### Kaggle score: 0.75718 rank: 1 username:shufan Xie

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import gaussian_process
from sklearn import linear_model
rf = RandomForestClassifier(n_estimators=1000, max_features=106, bootstrap=True, max_depth=10)
rf.fit(Xtr, Ytr)
pred_soft = rf.predict_proba(Xtr)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print(roc_auc)
pred_soft = rf.predict_proba(Xva)
false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
roc_auc = auc(false_positive_rate, true_positive_rate)
print (roc_auc)

0.9544672249953339
0.7470250188657339


In [96]:
max = 0
currk = 0

for mf in range(10,108):
    rf = RandomForestClassifier(n_estimators=50, max_features=mf, bootstrap=True, max_depth=20)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
    print ("maxF = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)

maxF = 10
Training: 1.0
Validation: 0.7155564303128811
maxF = 11
Training: 1.0
Validation: 0.7068863557206709
maxF = 12
Training: 1.0
Validation: 0.7205077784872584
maxF = 13
Training: 1.0
Validation: 0.7196098421083184
maxF = 14
Training: 0.9999992057913694
Validation: 0.7205621988738607
maxF = 15
Training: 1.0
Validation: 0.7243453227491726
maxF = 16
Training: 1.0
Validation: 0.7142539690601962
maxF = 17
Training: 1.0
Validation: 0.737723667788936
maxF = 18
Training: 0.9999996596248726
Validation: 0.7288096084634585
maxF = 19
Training: 0.9999999999999999
Validation: 0.7290399881000754
maxF = 20
Training: 1.0
Validation: 0.7368874078481453
maxF = 21
Training: 0.9999996596248725
Validation: 0.734501073895629
maxF = 22
Training: 1.0
Validation: 0.7254554986358622
maxF = 23
Training: 1.0
Validation: 0.7378297875428108
maxF = 24
Training: 0.999999432708121
Validation: 0.728073119231439
maxF = 25
Training: 0.9999997730832484
Validation: 0.734392233122424
maxF = 26
Training: 1.0
Validation:

In [97]:
max = 0
currk = 0

for mf in range(10,108):
    rf = RandomForestClassifier(n_estimators=100, max_features=mf, bootstrap=True, max_depth=20)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = mf
    
    print ("maxF = " + str(mf))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, mf)

maxF = 10
Training: 1.0
Validation: 0.713268053056249
maxF = 11
Training: 1.0
Validation: 0.7308730481221339
maxF = 12
Training: 0.9999999999999999
Validation: 0.7143020404016951
maxF = 13
Training: 1.0
Validation: 0.7352402841469785
maxF = 14
Training: 1.0
Validation: 0.7364964880710513
maxF = 15
Training: 1.0
Validation: 0.7307097869623266
maxF = 16
Training: 1.0
Validation: 0.7345536802693446
maxF = 17
Training: 1.0
Validation: 0.7399131813432402
maxF = 18
Training: 1.0
Validation: 0.7299569716143265
maxF = 19
Training: 1.0
Validation: 0.7339550560167178
maxF = 20
Training: 1.0
Validation: 0.7342607171881349
maxF = 21
Training: 1.0
Validation: 0.7322163246647705
maxF = 22
Training: 1.0
Validation: 0.7394279328960353
maxF = 23
Training: 1.0
Validation: 0.732518357810414
maxF = 24
Training: 1.0
Validation: 0.7385535786846229
maxF = 25
Training: 0.9999999999999999
Validation: 0.7408673521216695
maxF = 26
Training: 1.0
Validation: 0.7297973384802926
maxF = 27
Training: 1.0
Validation: 0

KeyboardInterrupt: 

In [ ]:
max = 0
currk = 0
depths=[10,20,30,40,50,60,70]
for d in depths:
    rf = RandomForestClassifier(n_estimators=100, max_features=106, bootstrap=True, max_depth=d)
    rf.fit(Xtr, Ytr)
    
    pred_soft = rf.predict_proba(Xtr)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytr, pred_soft[:,1])
    roc_auc_tr = auc(false_positive_rate, true_positive_rate)
    
    pred_soft = rf.predict_proba(Xva)
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Yva, pred_soft[:,1])
    roc_auc_va = auc(false_positive_rate, true_positive_rate)
    
    if roc_auc_va > max:
        max = roc_auc_va
        currk = d
    
    print ("depth = " + str(d))
    print ("Training: " + str(roc_auc_tr))
    print ("Validation: " + str(roc_auc_va))
    
print (max, currk)